## Preprocessing MIMIC

Now that we've explored the contents of the MIMIC database, let's put a heavier focus on the prediction tasks that we are trying to achieve:

* (1) mortality prediction
* (2) hospital readmission 
* (3) length of stay

We've seen in the previous part that the availble amount of features is extremely large. For that reason, we'll have to carefully select the columns and measurement types from the patient data in order to do our analysis. In this part, we'll dive into the feature selection and the preprocessing of the data so that it is usable for our machine learning classifiers.

## Features

We shall consider the following features:

* Age at admission
* Gender
* Admission type
* Vital signs
    * Heart rate
    * Body temperature
    * Respiratory rate
    * Mean and systolic blood pressure
    * SpO2

Outcomes:

* Length of ICU & hospital stay
* Hospital re-admission
* Mortality 30 days post-discharge


To consider:

* Diagnoses

In [2]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import getpass

# below imports are used to print out pretty pandas dataframes
from IPython.display import display, HTML

%matplotlib inline
plt.style.use('ggplot')

In [5]:
# SQL database config
sqluser = ''
dbname = 'MIMIC3'
schema_name = 'mimiciii'
hostname = ''
port = 5432
pwd = getpass.getpass()

········


In [65]:
# Connect to local postgres version of mimic
con = psycopg2.connect(dbname=dbname, user=sqluser, host=hostname, port=5432, password=pwd)
cur = con.cursor()
cur.execute('SET search_path to ' + schema_name)

## Exclusions

The following parameters will be taken into account to exclude a subset of the data.

* Patients aged less than 16 years old
    * This also removed neonates and children, which likely have different predictors than adults
* Second admissions
    * Simplifies analysis which assumes independent observations
    * We avoid taking into account that ICU stays are highly correlated
* Length of stay less than 2 days
    * helps remove false positives that we're placed in ICU for precaution
* Exclude patients based on hospital services
    * Patients undergoing surgery
    * Gets a more homogenous group of patients

In [85]:
query = \
"""
SET search_path to mimiciii;

WITH co AS
(
SELECT icu.subject_id, icu.hadm_id, icu.icustay_id, first_careunit
, icu.los as icu_los
, round((EXTRACT(EPOCH FROM (adm.dischtime-adm.admittime))/60/60/24) :: NUMERIC, 4) as hosp_los
, EXTRACT('epoch' from icu.intime - pat.dob) / 60.0 / 60.0 / 24.0 / 365.242 as age_icu_in
, pat.gender
, RANK() OVER (PARTITION BY icu.subject_id ORDER BY icu.intime) AS icustay_id_order
, hospital_expire_flag
, CASE WHEN pat.dod IS NOT NULL 
       AND pat.dod >= icu.intime - interval '6 hour'
       AND pat.dod <= icu.outtime + interval '6 hour' THEN 1 
       ELSE 0 END AS icu_expire_flag
FROM icustays icu
INNER JOIN patients pat
  ON icu.subject_id = pat.subject_id
INNER JOIN admissions adm
ON adm.hadm_id = icu.hadm_id    
)
, serv AS
(
SELECT icu.hadm_id, icu.icustay_id, se.curr_service
, CASE
    WHEN curr_service like '%SURG' then 1
    WHEN curr_service = 'ORTHO' then 1
    ELSE 0 END
  as surgical
, RANK() OVER (PARTITION BY icu.hadm_id ORDER BY se.transfertime DESC) as rank
FROM icustays icu
LEFT JOIN services se
 ON icu.hadm_id = se.hadm_id
AND se.transfertime < icu.intime + interval '12' hour
), excl AS
(
SELECT
  co.subject_id, co.hadm_id, co.icustay_id, co.icu_los, co.hosp_los
  , co.age_icu_in
  , co.gender
  , co.icustay_id_order
  , serv.curr_service
  , co.first_careunit
  , co.hospital_expire_flag
  , co.icu_expire_flag
  , CASE
        WHEN co.icu_los < 2 then 1
    ELSE 0 END
    AS exclusion_los
  , CASE
        WHEN co.age_icu_in < 16 then 1
    ELSE 0 END
    AS exclusion_age
  , CASE 
        WHEN co.icustay_id_order != 1 THEN 1
    ELSE 0 END 
    AS exclusion_first_stay
  , CASE
        WHEN serv.surgical = 1 THEN 1
    ELSE 0 END
    as exclusion_surgical
FROM co
LEFT JOIN serv
  ON  co.icustay_id = serv.icustay_id
  AND serv.rank = 1
)
SELECT *
FROM excl
"""
    
query_output = pd.read_sql_query(query,con)
query_output.head()    

,subject_id,hadm_id,icustay_id,icu_los,hosp_los,age_icu_in,gender,icustay_id_order,curr_service,first_careunit,hospital_expire_flag,icu_expire_flag,exclusion_los,exclusion_age,exclusion_first_stay,exclusion_surgical
0,2,163353,243653,0.0918,3.8639,0.002434,M,1,NB,NICU,0,0,1,1,0,0
1,3,145834,211552,6.0646,10.7847,76.526792,M,1,VSURG,MICU,0,0,0,0,0,1
2,4,185777,294638,1.6785,7.7590,47.845047,F,1,MED,MICU,0,0,1,0,0,0
3,5,178980,214757,0.0844,2.3222,0.000693,M,1,NB,NICU,0,0,1,1,0,0
4,6,107064,228232,3.6729,16.3646,65.942297,F,1,SURG,SICU,0,0,0,0,0,1


Let's now have a look at the proportion of the data that is excluded.

In [86]:
print('{:20s} {:5d}'.format('Observations', query_output.shape[0]))
idxExcl = np.zeros(query_output.shape[0],dtype=bool)
for col in query_output.columns:
    if "exclusion_" in col:
        print('{:20s} {:5d} ({:2.2f}%)'.format(col, query_output[col].sum(), query_output[col].sum()*100.0/query_output.shape[0]))
        idxExcl = (idxExcl) | (query_output[col]==1)

# print a summary of how many were excluded in total
print('')
print('{:20s} {:5d} ({:2.2f}%)'.format('Total excluded', np.sum(idxExcl), np.sum(idxExcl)*100.0/query_output.shape[0]))

Observations         61534
exclusion_los        29211 (47.47%)
exclusion_age         8109 (13.18%)
exclusion_first_stay 15058 (24.47%)
exclusion_surgical   18225 (29.62%)

Total excluded       48929 (79.52%)


Of the 61534 ICU stay observations, we exclude up to 48929 observations (almost 80% of samples), which is quite significant.

In [87]:
query = \
"""
SET search_path to mimiciii;

-- Table #1: Clinical data + demographics
WITH co AS
(
SELECT icu.subject_id, icu.hadm_id, icu.icustay_id, first_careunit
, icu.los as icu_los
, round((EXTRACT(EPOCH FROM (adm.dischtime-adm.admittime))/60/60/24) :: NUMERIC, 4) as hosp_los
, EXTRACT('epoch' from icu.intime - pat.dob) / 60.0 / 60.0 / 24.0 / 365.242 as age_icu_in
, pat.gender
, RANK() OVER (PARTITION BY icu.subject_id ORDER BY icu.intime) AS icustay_id_order
, hospital_expire_flag
, CASE WHEN pat.dod IS NOT NULL 
       AND pat.dod >= icu.intime - interval '6 hour'
       AND pat.dod <= icu.outtime + interval '6 hour' THEN 1 
       ELSE 0 END AS icu_expire_flag
FROM icustays icu
INNER JOIN patients pat
  ON icu.subject_id = pat.subject_id
INNER JOIN admissions adm
ON adm.hadm_id = icu.hadm_id    
)

-- Table #2: Services
, serv AS
(
SELECT icu.hadm_id, icu.icustay_id, se.curr_service
, CASE
    WHEN curr_service like '%SURG' then 1
    WHEN curr_service = 'ORTHO' then 1
    ELSE 0 END
  as surgical
, RANK() OVER (PARTITION BY icu.hadm_id ORDER BY se.transfertime DESC) as rank
FROM icustays icu
LEFT JOIN services se
 ON icu.hadm_id = se.hadm_id
AND se.transfertime < icu.intime + interval '12' hour
), excl AS
(
SELECT
  co.subject_id, co.hadm_id, co.icustay_id, co.icu_los, co.hosp_los
  , co.age_icu_in
  , co.gender
  , co.icustay_id_order
  , serv.curr_service
  , co.first_careunit
  , co.hospital_expire_flag
  , co.icu_expire_flag
  , CASE
        WHEN co.icu_los < 2 then 1
    ELSE 0 END
    AS exclusion_los
  , CASE
        WHEN co.age_icu_in < 16 then 1
    ELSE 0 END
    AS exclusion_age
  , CASE 
        WHEN co.icustay_id_order != 1 THEN 1
    ELSE 0 END 
    AS exclusion_first_stay
  , CASE
        WHEN serv.surgical = 1 THEN 1
    ELSE 0 END
    as exclusion_surgical
FROM co
LEFT JOIN serv
  ON  co.icustay_id = serv.icustay_id
  AND serv.rank = 1
)
SELECT subject_id, hadm_id, icustay_id, age_icu_in, first_careunit, gender, 
    hospital_expire_flag, icu_expire_flag, hosp_los, icu_los
FROM excl
WHERE exclusion_los = 0
AND exclusion_age = 0
AND exclusion_first_stay = 0
"""
    
query_output = pd.read_sql_query(query,con)
query_output.head()    

,subject_id,hadm_id,icustay_id,age_icu_in,first_careunit,gender,hospital_expire_flag,icu_expire_flag,hosp_los,icu_los
0,3,145834,211552,76.526792,MICU,M,0,0,10.7847,6.0646
1,6,107064,228232,65.942297,SICU,F,0,0,16.3646,3.6729
2,9,150750,220597,41.790228,MICU,M,1,1,4.8813,5.3231
3,12,112213,232669,72.374177,SICU,M,1,0,12.6958,7.6348
4,13,143045,263738,39.866118,CCU,F,0,0,6.8556,3.6660


Now, let's include additional information for each patient, more specifically physiological signs. We need to use the CHARTEVENTS table.

In [88]:
query = \
"""
-- Staging table #1: CHARTEVENTS
with ce as
(
  select adm.hadm_id
  , min(case when itemid in (211,220045) then valuenum else null end) as HeartRate_Min
  , max(case when itemid in (211,220045)  then valuenum else null end) as HeartRate_Max
  , min(case when itemid in (456,52,6702,443,220052,220181,225312) then valuenum else null end) as MeanBP_Min
  , max(case when itemid in (456,52,6702,443,220052,220181,225312)  then valuenum else null end) as MeanBP_Max
  , min(case when itemid in (615,618,220210,224690)  then valuenum else null end) as RespRate_Min
  , max(case when itemid in (615,618,220210,224690)  then valuenum else null end) as RespRate_Max
  from admissions adm
  left join chartevents chart
    on adm.hadm_id = chart.hadm_id
    and chart.itemid in
    (
    -- HEART RATE
    211, --"Heart Rate"
    220045, --"Heart Rate"

    -- MEAN BLOOD PRESSURE
    456, --"NBP Mean"
    52, --"Arterial BP Mean"
    6702, --	Arterial BP Mean #2
    443, --	Manual BP Mean(calc)
    220052, --"Arterial Blood Pressure mean"
    220181, --"Non Invasive Blood Pressure mean"
    225312, --"ART BP mean"

    -- RESPIRATORY RATE
    618,--	Respiratory Rate
    615,--	Resp Rate (Total)
    220210,--	Respiratory Rate
    224690 --	Respiratory Rate (Total)
    )
  left join icustays ie  
  -- match the tables on the patient identifier
  on ie.icustay_id = chart.icustay_id
  -- and require that the observation be made after the patient is admitted to the ICU
  and chart.charttime >= ie.intime
  -- and *before* their admission time + 1 day, i.e. the observation must be made on their first day in the ICU
  and chart.charttime <= ie.intime + interval '1 day'    
    group by adm.hadm_id
)

-- Table #2: Clinical data + demographics
, co AS
(
SELECT icu.subject_id, icu.hadm_id, icu.icustay_id, first_careunit
, icu.los as icu_los
, round((EXTRACT(EPOCH FROM (adm.dischtime-adm.admittime))/60/60/24) :: NUMERIC, 4) as hosp_los
, EXTRACT('epoch' from icu.intime - pat.dob) / 60.0 / 60.0 / 24.0 / 365.242 as age_icu_in
, pat.gender
, RANK() OVER (PARTITION BY icu.subject_id ORDER BY icu.intime) AS icustay_id_order
, hospital_expire_flag
, CASE WHEN pat.dod IS NOT NULL 
       AND pat.dod >= icu.intime - interval '6 hour'
       AND pat.dod <= icu.outtime + interval '6 hour' THEN 1 
       ELSE 0 END AS icu_expire_flag
FROM icustays icu
INNER JOIN patients pat
  ON icu.subject_id = pat.subject_id
INNER JOIN admissions adm
ON adm.hadm_id = icu.hadm_id    
)

-- Table #3: Services
, serv AS
(
SELECT icu.hadm_id, icu.icustay_id, se.curr_service
, CASE
    WHEN curr_service like '%SURG' then 1
    WHEN curr_service = 'ORTHO' then 1
    ELSE 0 END
  as surgical
, RANK() OVER (PARTITION BY icu.hadm_id ORDER BY se.transfertime DESC) as rank
FROM icustays icu
LEFT JOIN services se
 ON icu.hadm_id = se.hadm_id
AND se.transfertime < icu.intime + interval '12' hour
)
-- Table #4: Exclusions
, excl AS
(
SELECT
  co.subject_id, co.hadm_id, co.icustay_id, co.icu_los, co.hosp_los
  , co.age_icu_in
  , co.gender
  , co.icustay_id_order
  , serv.curr_service
  , co.first_careunit
  , co.hospital_expire_flag
  , co.icu_expire_flag
  , CASE
        WHEN co.icu_los < 2 then 1
    ELSE 0 END
    AS exclusion_los
  , CASE
        WHEN co.age_icu_in < 16 then 1
    ELSE 0 END
    AS exclusion_age
  , CASE 
        WHEN co.icustay_id_order != 1 THEN 1
    ELSE 0 END 
    AS exclusion_first_stay
  , CASE
        WHEN serv.surgical = 1 THEN 1
    ELSE 0 END
    as exclusion_surgical
FROM co
LEFT JOIN serv
  ON  co.icustay_id = serv.icustay_id
  AND serv.rank = 1
)

SELECT ex.subject_id, ex.hadm_id, ex.icustay_id, ex.age_icu_in, ex.first_careunit, ex.gender, 
    ex.hospital_expire_flag, ex.icu_expire_flag, ex.hosp_los, ex.icu_los

, ce.HeartRate_Min
, ce.HeartRate_Max
, ce.MeanBP_Min
, ce.MeanBP_Max
, ce.RespRate_Min
, ce.RespRate_Max

FROM excl ex
left join ce
  on ex.hadm_id = ce.hadm_id
WHERE exclusion_los = 0
AND exclusion_age = 0
AND exclusion_first_stay = 0;

"""

query_output = pd.read_sql_query(query,con)
query_output.head()

,subject_id,hadm_id,icustay_id,age_icu_in,first_careunit,gender,hospital_expire_flag,icu_expire_flag,hosp_los,icu_los,heartrate_min,heartrate_max,meanbp_min,meanbp_max,resprate_min,resprate_max
0,58526,100001,275225,35.476455,MICU,F,0,0,6.2076,4.2567,75.0,131.0,75.000000,131.000000,14.0,22.0
1,9895,100006,291788,48.917320,MICU,F,0,0,12.0618,4.9776,86.0,136.0,77.333298,128.667007,9.0,29.0
2,23018,100007,217937,73.823462,SICU,F,0,0,7.2965,4.0998,55.0,111.0,63.000000,129.000000,9.0,27.0
3,533,100009,253656,60.799222,CSRU,M,0,0,4.9035,2.4908,46.0,92.0,50.000000,82.000000,0.0,37.0
4,87977,100011,214619,21.504107,TSICU,M,0,0,14.3979,11.5029,68.0,142.0,-17.000000,150.000000,0.0,45.0
